# 🚀 DeepSeek-R1: Implementing Reasoning-Centric Reinforcement Learning

## 📚 Table of Contents
1. [Paper Implementation Overview](#overview)
2. [Reasoning-Centric Reward Design](#reward-design)
3. [PPO Implementation for Reasoning](#ppo-implementation)
4. [Reasoning Evaluation Framework](#evaluation)
5. [Training Pipeline](#training)
6. [Nigerian Context Applications](#nigerian-apps)

## 🎯 Implementation Objectives
- Implement DeepSeek-R1's reasoning-centric RL approach
- Build custom reward functions for reasoning quality
- Adapt PPO for reasoning tasks
- Create evaluation framework for reasoning capability
- Apply to Nigerian context use cases


## 1. Paper Implementation Overview <a id='overview'></a>

### 1.1 DeepSeek-R1 Architecture

```
┌───────────────────────────────────────────────────────────────┐
│                    DeepSeek-R1 System                          │
├─────────────────┬─────────────────┬─────────────────┬─────────┤
│  Pre-trained    │  Reasoning      │  PPO with       │  Reasoning│
│  Language Model │  Reward Model   │  Reasoning      │  Evaluation│
│  (Base LLM)     │  (Custom)       │  Adaptations    │  Framework │
└─────────────────┴─────────────────┴─────────────────┴─────────┘
                                      │
                                      ▼
┌───────────────────────────────────────────────────────────────┐
│                    Reasoning-Enhanced LLM                     │
└───────────────────────────────────────────────────────────────┘
```

**Key Components to Implement:**
1. **Reasoning Reward Model**: Custom reward functions
2. **PPO Adaptations**: Reasoning-specific modifications
3. **Evaluation Framework**: Multi-dimensional reasoning metrics
4. **Training Pipeline**: End-to-end reasoning optimization

**Implementation Roadmap:**
- Start with basic PPO implementation
- Add reasoning-specific reward functions
- Implement evaluation metrics
- Integrate with LLM policy
- Test on reasoning tasks


### 1.2 Required Libraries and Setup

Let's install the necessary libraries and set up our environment:

In [ ]:
# Install required libraries
%pip install torch numpy gymnasium matplotlib seaborn pandas scikit-learn
%pip install transformers datasets accelerate
%pip install stable-baselines3

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
import random
from typing import List, Dict, Tuple, Any

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

## 2. Reasoning-Centric Reward Design <a id='reward-design'></a>

### 2.1 Reasoning Reward Components

DeepSeek-R1 uses a composite reward function with multiple components:

$$
R_{total} = w_1 R_{answer} + w_2 R_{path} + w_3 R_{consistency} + w_4 R_{efficiency}
$$

Let's implement each component:

In [ ]:
class ReasoningRewardCalculator:
    """
    Implements the reasoning-centric reward function from DeepSeek-R1
    
    Components:
    - Answer Correctness (R_answer)
    - Path Quality (R_path)
    - Logical Consistency (R_consistency)
    - Efficiency (R_efficiency)
    """
    
    def __init__(self, 
                 w_answer: float = 1.0,
                 w_path: float = 0.8,
                 w_consistency: float = 0.6,
                 w_efficiency: float = 0.4):
        """
        Initialize reward calculator with component weights
        
        Args:
            w_answer: Weight for answer correctness
            w_path: Weight for reasoning path quality
            w_consistency: Weight for logical consistency
            w_efficiency: Weight for reasoning efficiency
        """
        self.weights = {
            'answer': w_answer,
            'path': w_path,
            'consistency': w_consistency,
            'efficiency': w_efficiency
        }
        
        # Normalize weights
        total_weight = sum(self.weights.values())
        self.weights = {k: v/total_weight for k, v in self.weights.items()}
    
    def calculate_answer_reward(self, answer: str, ground_truth: str) -> float:
        """
        Calculate reward for answer correctness
        
        Args:
            answer: Generated answer from LLM
            ground_truth: Correct answer
            
        Returns:
            Reward score (0.0 to 1.0)
        """
        # Simple exact match for now
        # In practice, would use semantic similarity, regex matching, etc.
        if answer.strip().lower() == ground_truth.strip().lower():
            return 1.0
        else:
            # Partial credit for similar answers
            return 0.0
    
    def calculate_path_quality(self, reasoning_steps: List[str]) -> float:
        """
        Calculate reward for reasoning path quality
        
        Args:
            reasoning_steps: List of reasoning steps
            
        Returns:
            Quality score (0.0 to 1.0)
        """
        if len(reasoning_steps) == 0:
            return 0.0
        
        # Simple heuristic: longer reasoning paths get higher quality scores
        # up to a point, then decrease (to prevent overly verbose reasoning)
        step_count = len(reasoning_steps)
        
        # Ideal step count range (3-7 steps typically good for reasoning)
        if step_count < 3:
            return step_count / 3  # Linear increase
        elif step_count <= 7:
            return 1.0  # Ideal range
        else:
            return max(0.0, 1.0 - (step_count - 7) * 0.1)  # Decrease for too many steps
    
    def calculate_consistency(self, reasoning_steps: List[str], answer: str) -> float:
        """
        Calculate reward for logical consistency
        
        Args:
            reasoning_steps: List of reasoning steps
            answer: Final answer
            
        Returns:
            Consistency score (0.0 to 1.0)
        """
        if len(reasoning_steps) == 0:
            return 1.0  # No steps to be inconsistent with
        
        # Simple consistency checks
        consistency_score = 1.0
        
        # Check if answer is mentioned in reasoning steps
        answer_in_steps = any(answer.lower() in step.lower() for step in reasoning_steps)
        if not answer_in_steps:
            consistency_score *= 0.7  # Penalty for answer not supported by reasoning
        
        # Check for obvious contradictions (simple implementation)
        # In practice, would use more sophisticated NLP techniques
        contradictions = self._check_contradictions(reasoning_steps)
        consistency_score *= (1.0 - 0.3 * contradictions)  # Penalty per contradiction
        
        return max(0.0, consistency_score)
    
    def _check_contradictions(self, reasoning_steps: List[str]) -> int:
        """
        Simple contradiction detection
        
        Returns:
            Number of contradictions detected
        """
        contradictions = 0
        
        # Check for obvious opposites
        opposite_pairs = [
            ('yes', 'no'), ('true', 'false'), ('correct', 'incorrect'),
            ('possible', 'impossible'), ('always', 'never'), ('all', 'none')
        ]
        
        for step in reasoning_steps:
            step_lower = step.lower()
            for pair in opposite_pairs:
                if pair[0] in step_lower and pair[1] in step_lower:
                    contradictions += 1
                    break  # Count each step only once
        
        return contradictions
    
    def calculate_efficiency(self, reasoning_steps: List[str]) -> float:
        """
        Calculate reward for reasoning efficiency
        
        Args:
            reasoning_steps: List of reasoning steps
            
        Returns:
            Efficiency score (0.0 to 1.0)
        """
        step_count = len(reasoning_steps)
        
        if step_count == 0:
            return 0.0
        
        # Efficiency: fewer steps are better, but not too few
        # Ideal range: 3-5 steps
        if step_count <= 3:
            return 1.0  # Very efficient
        elif step_count <= 5:
            return 0.8  # Good efficiency
        elif step_count <= 7:
            return 0.6  # Moderate efficiency
        else:
            return max(0.0, 0.4 - (step_count - 7) * 0.1)  # Decreasing efficiency
    
    def calculate_total_reward(self, 
                             reasoning_steps: List[str], 
                             answer: str, 
                             ground_truth: str) -> float:
        """
        Calculate the total reasoning reward
        
        Args:
            reasoning_steps: List of reasoning steps
            answer: Generated answer
            ground_truth: Correct answer
            
        Returns:
            Total weighted reward score
        """
        r_answer = self.calculate_answer_reward(answer, ground_truth)
        r_path = self.calculate_path_quality(reasoning_steps)
        r_consistency = self.calculate_consistency(reasoning_steps, answer)
        r_efficiency = self.calculate_efficiency(reasoning_steps)
        
        total_reward = (
            self.weights['answer'] * r_answer +
            self.weights['path'] * r_path +
            self.weights['consistency'] * r_consistency +
            self.weights['efficiency'] * r_efficiency
        )
        
        return total_reward
    
    def get_reward_components(self, 
                            reasoning_steps: List[str], 
                            answer: str, 
                            ground_truth: str) -> Dict[str, float]:
        """
        Get individual reward components for analysis
        
        Returns:
            Dictionary with all reward components
        """
        return {
            'answer': self.calculate_answer_reward(answer, ground_truth),
            'path': self.calculate_path_quality(reasoning_steps),
            'consistency': self.calculate_consistency(reasoning_steps, answer),
            'efficiency': self.calculate_efficiency(reasoning_steps),
            'total': self.calculate_total_reward(reasoning_steps, answer, ground_truth)
        }


In [ ]:
# Test the reward calculator
reward_calculator = ReasoningRewardCalculator()

# Example reasoning scenario
reasoning_steps = [
    "First, I need to understand the problem statement.",
    "The problem involves calculating the area of a rectangle.",
    "The formula for area is length × width.",
    "Given length = 5 and width = 3, so area = 5 × 3 = 15."
]
answer = "15"
ground_truth = "15"

# Calculate rewards
rewards = reward_calculator.get_reward_components(reasoning_steps, answer, ground_truth)

print("Reasoning Reward Components:")
for component, score in rewards.items():
    print(f"  {component.capitalize()}: {score:.3f}")
    
print(f"\nTotal Reasoning Reward: {rewards['total']:.3f}")

# Test with incorrect answer
wrong_answer = "25"
wrong_rewards = reward_calculator.get_reward_components(reasoning_steps, wrong_answer, ground_truth)
print(f"\nWith wrong answer '{wrong_answer}':")
print(f"  Answer Reward: {wrong_rewards['answer']:.3f}")
print(f"  Total Reward: {wrong_rewards['total']:.3f}")

### 2.2 Advanced Reward Functions

For more sophisticated reasoning evaluation, we can enhance our reward functions:

In [ ]:
class AdvancedReasoningReward(ReasoningRewardCalculator):
    """
    Enhanced reasoning reward calculator with more sophisticated metrics
    """
    
    def __init__(self):
        super().__init__()
        # Add additional metrics
        self.weights['novelty'] = 0.3
        self.weights['depth'] = 0.5
        
        # Renormalize weights
        total_weight = sum(self.weights.values())
        self.weights = {k: v/total_weight for k, v in self.weights.items()}
    
    def calculate_novelty(self, reasoning_steps: List[str]) -> float:
        """
        Calculate novelty reward - encourages creative but valid reasoning paths
        """
        if len(reasoning_steps) == 0:
            return 0.0
        
        # Simple heuristic: longer unique steps indicate more novelty
        unique_steps = len(set(reasoning_steps))
        total_steps = len(reasoning_steps)
        
        # Novelty score based on uniqueness ratio
        uniqueness_ratio = unique_steps / total_steps
        
        # Also consider step diversity (vocabulary richness)
        all_words = ' '.join(reasoning_steps).split()
        unique_words = len(set(all_words))
        vocabulary_richness = min(1.0, unique_words / len(all_words))
        
        return 0.5 * uniqueness_ratio + 0.5 * vocabulary_richness
    
    def calculate_depth(self, reasoning_steps: List[str]) -> float:
        """
        Calculate reasoning depth - measures complexity and multi-step reasoning
        """
        if len(reasoning_steps) == 0:
            return 0.0
        
        # Count logical connectors and mathematical operations
        logical_indicators = ['therefore', 'thus', 'hence', 'so', 'because',
                            'since', 'if', 'then', 'implies', 'follows',
                            '+', '-', '×', '*', '÷', '/', '=', '≠', '<', '>',
                            'sum', 'product', 'difference', 'quotient']
        
        depth_score = 0.0
        for step in reasoning_steps:
            step_lower = step.lower()
            # Count logical indicators
            indicators_found = sum(1 for indicator in logical_indicators 
                                 if indicator in step_lower)
            depth_score += indicators_found
        
        # Normalize by step count
        depth_score = depth_score / len(reasoning_steps)
        
        # Scale to 0-1 range
        return min(1.0, depth_score / 3.0)  # Cap at reasonable depth
    
    def calculate_total_reward(self, 
                             reasoning_steps: List[str], 
                             answer: str, 
                             ground_truth: str) -> float:
        """
        Calculate total reward with advanced metrics
        """
        # Get basic rewards
        basic_rewards = super().get_reward_components(reasoning_steps, answer, ground_truth)
        
        # Add advanced metrics
        r_novelty = self.calculate_novelty(reasoning_steps)
        r_depth = self.calculate_depth(reasoning_steps)
        
        total_reward = (
            self.weights['answer'] * basic_rewards['answer'] +
            self.weights['path'] * basic_rewards['path'] +
            self.weights['consistency'] * basic_rewards['consistency'] +
            self.weights['efficiency'] * basic_rewards['efficiency'] +
            self.weights['novelty'] * r_novelty +
            self.weights['depth'] * r_depth
        )
        
        return total_reward
    
    def get_reward_components(self, 
                            reasoning_steps: List[str], 
                            answer: str, 
                            ground_truth: str) -> Dict[str, float]:
        """
        Get all reward components including advanced metrics
        """
        basic_components = super().get_reward_components(reasoning_steps, answer, ground_truth)
        
        return {
            **basic_components,
            'novelty': self.calculate_novelty(reasoning_steps),
            'depth': self.calculate_depth(reasoning_steps)
        }


In [ ]:
# Test advanced reward calculator
advanced_reward = AdvancedReasoningReward()

# More complex reasoning example
complex_steps = [
    "Let's analyze this mathematical problem step by step.",
    "First, I need to identify the variables: x represents the unknown quantity.",
    "The equation given is 2x + 5 = 15, which is a linear equation.",
    "To solve for x, I should isolate it on one side of the equation.",
    "Subtracting 5 from both sides gives: 2x = 10.",
    "Then, dividing both sides by 2 yields: x = 5.",
    "Therefore, the solution to the equation is x = 5."
]

complex_rewards = advanced_reward.get_reward_components(complex_steps, "x = 5", "x = 5")

print("Advanced Reasoning Reward Components:")
for component, score in complex_rewards.items():
    print(f"  {component.capitalize()}: {score:.3f}")
    
print(f"\nTotal Advanced Reward: {complex_rewards['total']:.3f}")

# Compare with simple reasoning
simple_steps = ["The answer is 5."]
simple_rewards = advanced_reward.get_reward_components(simple_steps, "5", "5")
print(f"\nSimple reasoning total reward: {simple_rewards['total']:.3f}")
print(f"Complex reasoning total reward: {complex_rewards['total']:.3f}")
print(f"Difference: {complex_rewards['total'] - simple_rewards['total']:.3f}")

## 3. PPO Implementation for Reasoning <a id='ppo-implementation'></a>

### 3.1 PPO for Reasoning Tasks

Now let's implement Proximal Policy Optimization adapted for reasoning tasks:

In [ ]:
class ReasoningPPO:
    """
    Proximal Policy Optimization for Reasoning Tasks
    
    Adapted from DeepSeek-R1 approach with reasoning-specific modifications
    """
    
    def __init__(self, 
                 policy_network: torch.nn.Module,
                 value_network: torch.nn.Module,
                 lr: float = 3e-4,
                 gamma: float = 0.99,
                 clip_epsilon: float = 0.2,
                 ppo_epochs: int = 4,
                 batch_size: int = 64,
                 reward_calculator: ReasoningRewardCalculator = None):
        """
        Initialize PPO for reasoning
        
        Args:
            policy_network: Neural network for policy (LLM in our case)
            value_network: Neural network for value function
            lr: Learning rate
            gamma: Discount factor
            clip_epsilon: Clipping parameter for PPO
            ppo_epochs: Number of optimization epochs per batch
            batch_size: Mini-batch size
            reward_calculator: Reasoning reward calculator
        """
        self.policy = policy_network.to(device)
        self.value = value_network.to(device)
        
        self.policy_optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr)
        self.value_optimizer = torch.optim.Adam(self.value.parameters(), lr=lr)
        
        self.gamma = gamma
        self.clip_epsilon = clip_epsilon
        self.ppo_epochs = ppo_epochs
        self.batch_size = batch_size
        
        # Reasoning-specific components
        self.reward_calculator = reward_calculator or ReasoningRewardCalculator()
        
        # Memory for storing trajectories
        self.memory = {
            'states': [],
            'actions': [],
            'log_probs': [],
            'rewards': [],
            'dones': [],
            'values': []
        }
    
    def store_transition(self, 
                        state: Any,
                        action: Any,
                        log_prob: torch.Tensor,
                        reward: float,
                        done: bool,
                        value: float):
        """
        Store transition in memory
        """
        self.memory['states'].append(state)
        self.memory['actions'].append(action)
        self.memory['log_probs'].append(log_prob)
        self.memory['rewards'].append(reward)
        self.memory['dones'].append(done)
        self.memory['values'].append(value)
    
    def compute_returns(self, rewards: List[float], dones: List[bool]) -> List[float]:
        """
        Compute discounted returns
        """
        returns = []
        discounted_sum = 0
        
        for reward, done in zip(reversed(rewards), reversed(dones)):
            if done:
                discounted_sum = 0
            discounted_sum = reward + self.gamma * discounted_sum
            returns.insert(0, discounted_sum)
        
        return returns
    
    def compute_advantages(self, rewards: List[float], 
                          dones: List[bool], 
                          values: List[float]) -> torch.Tensor:
        """
        Compute advantages using Generalized Advantage Estimation (GAE)
        """
        returns = self.compute_returns(rewards, dones)
        
        # Convert to tensors
        returns_tensor = torch.tensor(returns, dtype=torch.float32, device=device)
        values_tensor = torch.tensor(values, dtype=torch.float32, device=device)
        
        # Advantages = Returns - Values
        advantages = returns_tensor - values_tensor
        
        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        return advantages
    
    def ppo_update(self):
        """
        Perform PPO update using stored trajectories
        """
        if len(self.memory['states']) == 0:
            return 0, 0, 0  # No data to update
        
        # Convert memory to tensors
        old_states = self.memory['states']
        old_actions = self.memory['actions']
        old_log_probs = torch.cat(self.memory['log_probs']).detach()
        old_values = torch.tensor(self.memory['values'], dtype=torch.float32, device=device)
        
        # Compute advantages
        advantages = self.compute_advantages(
            self.memory['rewards'], 
            self.memory['dones'], 
            self.memory['values']
        )
        
        # Compute returns
        returns = torch.tensor(
            self.compute_returns(self.memory['rewards'], self.memory['dones']),
            dtype=torch.float32, 
            device=device
        )
        
        # Clear memory
        self.memory = {k: [] for k in self.memory.keys()}
        
        # Number of updates
        num_updates = len(old_states) // self.batch_size
        
        # PPO optimization loop
        for epoch in range(self.ppo_epochs):
            for i in range(num_updates):
                # Get mini-batch indices
                start_idx = i * self.batch_size
                end_idx = start_idx + self.batch_size
                
                # Get mini-batch data
                batch_indices = list(range(start_idx, end_idx))
                
                # Get current policy outputs
                batch_states = [old_states[idx] for idx in batch_indices]
                batch_actions = [old_actions[idx] for idx in batch_indices]
                
                # Compute current log probabilities
                dist = self.policy(batch_states)
                new_log_probs = dist.log_prob(batch_actions)
                
                # Compute probability ratio
                ratio = torch.exp(new_log_probs - old_log_probs[batch_indices])
                
                # Compute clipped surrogate objective
                surr1 = ratio * advantages[batch_indices]
                surr2 = torch.clamp(ratio, 
                                   1.0 - self.clip_epsilon, 
                                   1.0 + self.clip_epsilon) * advantages[batch_indices]
                
                policy_loss = -torch.min(surr1, surr2).mean()
                
                # Value function loss
                value_loss = torch.nn.functional.mse_loss(
                    self.value(batch_states).squeeze(), 
                    returns[batch_indices]
                )
                
                # Total loss
                loss = policy_loss + 0.5 * value_loss
                
                # Update networks
                self.policy_optimizer.zero_grad()
                self.value_optimizer.zero_grad()
                loss.backward()
                
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(self.policy.parameters(), 0.5)
                torch.nn.utils.clip_grad_norm_(self.value.parameters(), 0.5)
                
                self.policy_optimizer.step()
                self.value_optimizer.step()
        
        return policy_loss.item(), value_loss.item(), loss.item()


### 3.2 Reasoning-Specific PPO Adaptations

Let's create an enhanced version with reasoning-specific features:

In [ ]:
class DeepSeekR1PPO(ReasoningPPO):
    """
    DeepSeek-R1 specific PPO implementation with reasoning enhancements
    """
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Reasoning-specific parameters
        self.reasoning_regularization = 0.1  # Strength of reasoning structure regularization
        self.curriculum_learning = True      # Enable curriculum learning
        self.reasoning_difficulty = 1.0      # Current difficulty level
        
        # Reasoning structure metrics
        self.reasoning_metrics = {
            'avg_steps': deque(maxlen=100),
            'consistency': deque(maxlen=100),
            'depth': deque(maxlen=100),
            'novelty': deque(maxlen=100)
        }
    
    def reasoning_structure_loss(self, reasoning_steps: List[str]) -> torch.Tensor:
        """
        Compute reasoning structure regularization loss
        
        Encourages well-structured, logical reasoning patterns
        """
        if not reasoning_steps:
            return torch.tensor(0.0, device=device)
        
        # Convert reasoning steps to tensor representation
        # This is a simplified version - in practice would use embeddings
        step_tensors = []
        for step in reasoning_steps:
            # Simple feature extraction
            features = [
                len(step.split()),  # Step length
                step.count('because'),  # Causal reasoning indicators
                step.count('therefore'),
                step.count('so'),
                step.count('='),  # Mathematical reasoning
                step.count('+') + step.count('-') + step.count('*') + step.count('/')
            ]
            step_tensors.append(torch.tensor(features, dtype=torch.float32, device=device))
        
        if len(step_tensors) == 0:
            return torch.tensor(0.0, device=device)
        
        # Stack all step tensors
        reasoning_tensor = torch.stack(step_tensors)
        
        # Compute structure metrics
        # 1. Step consistency (similarity between consecutive steps)
        if len(reasoning_tensor) > 1:
            step_diffs = torch.norm(reasoning_tensor[1:] - reasoning_tensor[:-1], dim=1)
            consistency_loss = torch.mean(step_diffs)  # Encourage smooth transitions
        else:
            consistency_loss = torch.tensor(0.0, device=device)
        
        # 2. Reasoning depth (complexity)
        depth_features = reasoning_tensor[:, [1, 2, 3, 4, 5]]  # Reasoning indicators
        depth_score = torch.sum(depth_features)
        depth_loss = 1.0 / (1.0 + depth_score)  # Encourage more reasoning indicators
        
        # 3. Step diversity
        unique_features = torch.unique(reasoning_tensor, dim=0)
        diversity_ratio = len(unique_features) / len(reasoning_tensor)
        diversity_loss = 1.0 - diversity_ratio  # Encourage diverse reasoning steps
        
        # Combine structure losses
        total_structure_loss = (
            0.4 * consistency_loss +
            0.3 * depth_loss +
            0.3 * diversity_loss
        )
        
        return total_structure_loss
    
    def curriculum_learning_adjustment(self) -> float:
        """
        Adjust reasoning difficulty based on current performance
        """
        if not self.curriculum_learning:
            return self.reasoning_difficulty
        
        # Simple curriculum: increase difficulty based on recent performance
        if len(self.reasoning_metrics['avg_steps']) > 10:
            avg_steps = np.mean(self.reasoning_metrics['avg_steps'])
            avg_consistency = np.mean(self.reasoning_metrics['consistency'])
            
            # If doing well (good consistency, reasonable step count), increase difficulty
            if avg_consistency > 0.8 and 3 <= avg_steps <= 7:
                self.reasoning_difficulty = min(5.0, self.reasoning_difficulty + 0.1)
            # If struggling, decrease difficulty
            elif avg_consistency < 0.5 or avg_steps > 10:
                self.reasoning_difficulty = max(0.5, self.reasoning_difficulty - 0.1)
        
        return self.reasoning_difficulty
    
    def update_reasoning_metrics(self, reasoning_steps: List[str], answer: str, ground_truth: str):
        """
        Update reasoning performance metrics
        """
        # Use advanced reward calculator for detailed metrics
        if not hasattr(self, 'advanced_reward'):
            self.advanced_reward = AdvancedReasoningReward()
        
        metrics = self.advanced_reward.get_reward_components(reasoning_steps, answer, ground_truth)
        
        # Update metrics
        self.reasoning_metrics['avg_steps'].append(len(reasoning_steps))
        self.reasoning_metrics['consistency'].append(metrics['consistency'])
        self.reasoning_metrics['depth'].append(metrics['depth'])
        self.reasoning_metrics['novelty'].append(metrics['novelty'])
    
    def get_reasoning_performance(self) -> Dict[str, float]:
        """
        Get current reasoning performance metrics
        """
        return {
            'avg_steps': np.mean(self.reasoning_metrics['avg_steps']) if self.reasoning_metrics['avg_steps'] else 0,
            'avg_consistency': np.mean(self.reasoning_metrics['consistency']) if self.reasoning_metrics['consistency'] else 0,
            'avg_depth': np.mean(self.reasoning_metrics['depth']) if self.reasoning_metrics['depth'] else 0,
            'avg_novelty': np.mean(self.reasoning_metrics['novelty']) if self.reasoning_metrics['novelty'] else 0,
            'current_difficulty': self.reasoning_difficulty
        }
    
    def ppo_update(self):
        """
        DeepSeek-R1 enhanced PPO update with reasoning-specific features
        """
        if len(self.memory['states']) == 0:
            return 0, 0, 0, 0  # No data to update
        
        # Get reasoning difficulty for this batch
        difficulty = self.curriculum_learning_adjustment()
        
        # Standard PPO update
        policy_loss, value_loss, total_loss = super().ppo_update()
        
        # Add reasoning structure regularization
        reasoning_loss = 0.0
        reasoning_count = 0
        
        # Process reasoning steps from memory
        for i, state in enumerate(self.memory['states']):
            # Extract reasoning steps from state
            # In practice, state would contain reasoning history
            reasoning_steps = state.get('reasoning_steps', [])
            if reasoning_steps:
                structure_loss = self.reasoning_structure_loss(reasoning_steps)
                reasoning_loss += structure_loss.item()
                reasoning_count += 1
                
                # Update reasoning metrics
                action = self.memory['actions'][i]
                # In practice, action would contain the final answer
                answer = action.get('answer', '') if isinstance(action, dict) else str(action)
                ground_truth = state.get('ground_truth', '')
                self.update_reasoning_metrics(reasoning_steps, answer, ground_truth)
        
        if reasoning_count > 0:
            reasoning_loss /= reasoning_count
        
        # Total loss with reasoning regularization
        total_loss_with_reasoning = total_loss + self.reasoning_regularization * reasoning_loss
        
        return policy_loss, value_loss, reasoning_loss, total_loss_with_reasoning


## 4. Reasoning Evaluation Framework <a id='evaluation'></a>

### 4.1 Comprehensive Reasoning Evaluation

Let's implement a comprehensive evaluation framework for reasoning capability:

In [ ]:
class ReasoningEvaluator:
    """
    Comprehensive evaluation framework for reasoning capability
    
    Implements multiple evaluation dimensions as used in DeepSeek-R1
    """
    
    def __init__(self):
        self.reward_calculator = AdvancedReasoningReward()
        self.evaluation_history = []
    
    def evaluate_reasoning(self, 
                          reasoning_steps: List[str], 
                          answer: str, 
                          ground_truth: str, 
                          problem_type: str = "general") -> Dict[str, Any]:
        """
        Comprehensive evaluation of reasoning capability
        
        Args:
            reasoning_steps: List of reasoning steps
            answer: Generated answer
            ground_truth: Correct answer
            problem_type: Type of reasoning problem
            
        Returns:
            Dictionary with comprehensive evaluation metrics
        """
        # Calculate reward components
        rewards = self.reward_calculator.get_reward_components(
            reasoning_steps, answer, ground_truth
        )
        
        # Additional evaluation metrics
        evaluation = {
            'problem_type': problem_type,
            'step_count': len(reasoning_steps),
            'answer_correct': rewards['answer'] == 1.0,
            'rewards': rewards,
            'reasoning_quality': self._calculate_reasoning_quality(reasoning_steps),
            'logical_structure': self._evaluate_logical_structure(reasoning_steps),
            'mathematical_rigor': self._evaluate_mathematical_rigor(reasoning_steps),
            'creativity': self._evaluate_creativity(reasoning_steps),
            'overall_score': self._calculate_overall_score(rewards)
        }
        
        # Store evaluation history
        self.evaluation_history.append(evaluation)
        
        return evaluation
    
    def _calculate_reasoning_quality(self, reasoning_steps: List[str]) -> float:
        """
        Calculate overall reasoning quality score
        """
        if len(reasoning_steps) == 0:
            return 0.0
        
        # Quality indicators
        quality_score = 0.0
        
        # Check for clear problem understanding
        has_problem_understanding = any(
            'problem' in step.lower() or 
            'question' in step.lower() or
            'task' in step.lower()
            for step in reasoning_steps
        )
        
        # Check for methodical approach
        has_methodical_approach = any(
            'first' in step.lower() or
            'next' in step.lower() or
            'then' in step.lower() or
            'step' in step.lower()
            for step in reasoning_steps
        )
        
        # Check for conclusion
        has_conclusion = any(
            'therefore' in step.lower() or
            'thus' in step.lower() or
            'so' in step.lower() or
            'conclude' in step.lower() or
            'answer' in step.lower()
            for step in reasoning_steps
        )
        
        # Calculate quality score
        quality_score += 0.4 if has_problem_understanding else 0.0
        quality_score += 0.3 if has_methodical_approach else 0.0
        quality_score += 0.3 if has_conclusion else 0.0
        
        return quality_score
    
    def _evaluate_logical_structure(self, reasoning_steps: List[str]) -> Dict[str, float]:
        """
        Evaluate logical structure of reasoning
        """
        if len(reasoning_steps) == 0:
            return {'coherence': 0.0, 'flow': 0.0, 'completeness': 0.0}
        
        # Coherence: how well steps connect to each other
        coherence = self._calculate_coherence(reasoning_steps)
        
        # Flow: smooth progression from problem to solution
        flow = self._calculate_flow(reasoning_steps)
        
        # Completeness: covers all necessary aspects
        completeness = self._calculate_completeness(reasoning_steps)
        
        return {
            'coherence': coherence,
            'flow': flow,
            'completeness': completeness,
            'structure_score': 0.4 * coherence + 0.3 * flow + 0.3 * completeness
        }
    
    def _calculate_coherence(self, reasoning_steps: List[str]) -> float:
        """
        Calculate coherence between reasoning steps
        """
        if len(reasoning_steps) <= 1:
            return 1.0
        
        # Simple coherence metric: check for connecting words
        connecting_words = ['therefore', 'thus', 'so', 'because', 'since', 
                          'hence', 'consequently', 'as a result', 'this means']
        
        connections = 0
        for i in range(1, len(reasoning_steps)):
            step = reasoning_steps[i].lower()
            if any(word in step for word in connecting_words):
                connections += 1
        
        return connections / (len(reasoning_steps) - 1)
    
    def _calculate_flow(self, reasoning_steps: List[str]) -> float:
        """
        Calculate flow from problem to solution
        """
        if len(reasoning_steps) == 0:
            return 0.0
        
        # Check if first step mentions problem/question
        first_step = reasoning_steps[0].lower()
        mentions_problem = any(word in first_step for word in 
                              ['problem', 'question', 'task', 'issue', 'challenge'])
        
        # Check if last step mentions solution/answer
        last_step = reasoning_steps[-1].lower()
        mentions_solution = any(word in last_step for word in 
                               ['solution', 'answer', 'result', 'conclusion', 'therefore'])
        
        # Check if middle steps contain reasoning
        middle_steps = reasoning_steps[1:-1] if len(reasoning_steps) > 2 else []
        has_reasoning = any(any(word in step.lower() for word in 
                               ['because', 'since', 'if', 'then', 'so', 'thus'])
                          for step in middle_steps) if middle_steps else False
        
        flow_score = 0.0
        flow_score += 0.4 if mentions_problem else 0.0
        flow_score += 0.4 if mentions_solution else 0.0
        flow_score += 0.2 if has_reasoning else 0.0
        
        return flow_score
    
    def _calculate_completeness(self, reasoning_steps: List[str]) -> float:
        """
        Calculate completeness of reasoning
        """
        if len(reasoning_steps) == 0:
            return 0.0
        
        # Simple heuristic: reasonable number of steps indicates completeness
        step_count = len(reasoning_steps)
        
        if step_count < 3:
            return 0.5  # Likely incomplete
        elif step_count <= 7:
            return 1.0  # Likely complete
        else:
            return max(0.0, 1.0 - (step_count - 7) * 0.05)  # May be overly verbose
    
    def _evaluate_mathematical_rigor(self, reasoning_steps: List[str]) -> float:
        """
        Evaluate mathematical rigor in reasoning
        """
        if len(reasoning_steps) == 0:
            return 0.0
        
        # Count mathematical elements
        math_indicators = ['=', '+', '-', '×', '*', '÷', '/', '^', '√', '%',
                          'sum', 'product', 'difference', 'quotient', 'square',
                          'equation', 'formula', 'calculate', 'compute', 'solve']
        
        math_count = 0
        for step in reasoning_steps:
            step_lower = step.lower()
            math_count += sum(1 for indicator in math_indicators 
                            if indicator in step_lower)
        
        # Normalize by step count
        math_density = math_count / len(reasoning_steps)
        
        # Scale to 0-1 range
        return min(1.0, math_density / 2.0)
    
    def _evaluate_creativity(self, reasoning_steps: List[str]) -> float:
        """
        Evaluate creativity in reasoning approach
        """
        if len(reasoning_steps) == 0:
            return 0.0
        
        # Creativity indicators
        creative_words = ['alternative', 'another', 'different', 'novel', 
                         'unique', 'creative', 'innovative', 'unconventional',
                         'approach', 'method', 'way', 'perspective', 'view']
        
        creativity_score = 0.0
        
        # Check for multiple approaches
        has_alternatives = any('alternative' in step.lower() or 
                              'another' in step.lower() or
                              'different approach' in step.lower()
                              for step in reasoning_steps)
        
        # Check for novel connections
        has_novel_connections = any('unexpected' in step.lower() or
                                   'surprising' in step.lower() or
                                   'interesting' in step.lower()
                                   for step in reasoning_steps)
        
        # Check for diverse vocabulary
        all_words = ' '.join(reasoning_steps).split()
        unique_words = len(set(all_words))
        vocabulary_diversity = unique_words / len(all_words) if all_words else 0
        
        creativity_score += 0.4 if has_alternatives else 0.0
        creativity_score += 0.3 if has_novel_connections else 0.0
        creativity_score += 0.3 * vocabulary_diversity
        
        return creativity_score
    
    def _calculate_overall_score(self, rewards: Dict[str, float]) -> float:
        """
        Calculate overall reasoning score
        """
        # Weighted combination of key metrics
        overall_score = (
            0.4 * rewards['answer'] +      # Answer correctness
            0.2 * rewards['consistency'] + # Logical consistency
            0.15 * rewards['path'] +      # Path quality
            0.1 * rewards['efficiency'] +  # Efficiency
            0.1 * rewards['depth'] +       # Reasoning depth
            0.05 * rewards['novelty']      # Creativity
        )
        
        return overall_score
    
    def get_evaluation_summary(self) -> Dict[str, Any]:
        """
        Get summary statistics from evaluation history
        """
        if not self.evaluation_history:
            return {}
        
        # Calculate averages
        avg_answer_correct = np.mean([eval['answer_correct'] for eval in self.evaluation_history])
        avg_overall_score = np.mean([eval['overall_score'] for eval in self.evaluation_history])
        avg_step_count = np.mean([eval['step_count'] for eval in self.evaluation_history])
        avg_reasoning_quality = np.mean([eval['reasoning_quality'] for eval in self.evaluation_history])
        avg_structure_score = np.mean([eval['logical_structure']['structure_score'] 
                                     for eval in self.evaluation_history])
        
        return {
            'total_evaluations': len(self.evaluation_history),
            'avg_answer_correct': avg_answer_correct,
            'avg_overall_score': avg_overall_score,
            'avg_step_count': avg_step_count,
            'avg_reasoning_quality': avg_reasoning_quality,
            'avg_structure_score': avg_structure_score,
            'problem_type_distribution': self._get_problem_type_distribution()
        }
    
    def _get_problem_type_distribution(self) -> Dict[str, int]:
        """
        Get distribution of problem types
        """
        distribution = {}
        for eval in self.evaluation_history:
            problem_type = eval['problem_type']
            distribution[problem_type] = distribution.get(problem_type, 0) + 1
        return distribution
    
    def plot_evaluation_metrics(self):
        """
        Plot evaluation metrics over time
        """
        if not self.evaluation_history:
            print("No evaluation history to plot.")
            return
        
        # Extract metrics over time
        overall_scores = [eval['overall_score'] for eval in self.evaluation_history]
        answer_correct = [float(eval['answer_correct']) for eval in self.evaluation_history]
        step_counts = [eval['step_count'] for eval in self.evaluation_history]
        
        plt.figure(figsize=(15, 10))
        
        # Overall score
        plt.subplot(2, 2, 1)
        plt.plot(overall_scores, label='Overall Score')
        plt.title('Overall Reasoning Score Over Time')
        plt.xlabel('Evaluation')
        plt.ylabel('Score')
        plt.ylim(0, 1)
        plt.grid(True)
        
        # Answer correctness
        plt.subplot(2, 2, 2)
        plt.plot(answer_correct, label='Answer Correct', color='green')
        plt.title('Answer Correctness Over Time')
        plt.xlabel('Evaluation')
        plt.ylabel('Correct (1.0) / Incorrect (0.0)')
        plt.ylim(0, 1.1)
        plt.grid(True)
        
        # Step count
        plt.subplot(2, 2, 3)
        plt.plot(step_counts, label='Step Count', color='purple')
        plt.title('Reasoning Step Count Over Time')
        plt.xlabel('Evaluation')
        plt.ylabel('Number of Steps')
        plt.grid(True)
        
        # Reasoning quality
        plt.subplot(2, 2, 4)
        reasoning_qualities = [eval['reasoning_quality'] for eval in self.evaluation_history]
        plt.plot(reasoning_qualities, label='Reasoning Quality', color='orange')
        plt.title('Reasoning Quality Over Time')
        plt.xlabel('Evaluation')
        plt.ylabel('Quality Score')
        plt.ylim(0, 1)
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()


In [ ]:
# Test the reasoning evaluator
evaluator = ReasoningEvaluator()

# Test with different reasoning examples
examples = [
    {
        'name': 'Simple Math',
        'steps': [
            'I need to calculate 5 + 3.',
            '5 + 3 equals 8.',
            'So the answer is 8.'
        ],
        'answer': '8',
        'ground_truth': '8',
        'type': 'math'
    },
    {
        'name': 'Complex Reasoning',
        'steps': [
            'Let me analyze this problem systematically.',
            'First, I need to understand the variables involved.',
            'The equation 2x + 5 = 15 can be solved step by step.',
            'Subtracting 5 from both sides gives 2x = 10.',
            'Then dividing by 2 gives x = 5.',
            'Therefore, the solution is x = 5.'
        ],
        'answer': 'x = 5',
        'ground_truth': 'x = 5',
        'type': 'math'
    },
    {
        'name': 'Logical Reasoning',
        'steps': [
            'All humans are mortal.',
            'Socrates is a human.',
            'Therefore, Socrates is mortal.'
        ],
        'answer': 'Socrates is mortal',
        'ground_truth': 'Socrates is mortal',
        'type': 'logic'
    }
]

# Evaluate all examples
for example in examples:
    print(f"\n=== {example['name']} ===")
    evaluation = evaluator.evaluate_reasoning(
        example['steps'], 
        example['answer'], 
        example['ground_truth'], 
        example['type']
    )
    
    print(f"Answer Correct: {evaluation['answer_correct']}")
    print(f"Overall Score: {evaluation['overall_score']:.3f}")
    print(f"Reasoning Quality: {evaluation['reasoning_quality']:.3f}")
    print(f"Structure Score: {evaluation['logical_structure']['structure_score']:.3f}")
    print(f"Mathematical Rigor: {evaluation['mathematical_rigor']:.3f}")
    print(f"Creativity: {evaluation['creativity']:.3f}")

# Show summary
print("\n=== Evaluation Summary ===")
summary = evaluator.get_evaluation_summary()
for key, value in summary.items():
    if isinstance(value, dict):
        print(f"{key}: {value}")
    else:
        print(f"{key}: {value:.3f}")

# Plot metrics
evaluator.plot_evaluation_metrics()

## 5. Training Pipeline <a id='training'></a>

### 5.1 Complete Training Pipeline

Now let's implement the complete training pipeline for DeepSeek-R1:

In [ ]:
class DeepSeekR1Trainer:
    """
    Complete training pipeline for DeepSeek-R1
    
    Implements the full reasoning-centric RL training process
    """
    
    def __init__(self, 
                 ppo_agent: DeepSeekR1PPO,
                 evaluator: ReasoningEvaluator,
                 max_episodes: int = 100,
                 max_steps_per_episode: int = 20):
        """
        Initialize DeepSeek-R1 trainer
        
        Args:
            ppo_agent: DeepSeek-R1 PPO agent
            evaluator: Reasoning evaluator
            max_episodes: Maximum number of training episodes
            max_steps_per_episode: Maximum steps per episode
        """
        self.ppo_agent = ppo_agent
        self.evaluator = evaluator
        self.max_episodes = max_episodes
        self.max_steps_per_episode = max_steps_per_episode
        
        # Training metrics
        self.training_metrics = {
            'episode_rewards': [],
            'policy_losses': [],
            'value_losses': [],
            'reasoning_losses': [],
            'total_losses': [],
            'answer_accuracy': [],
            'reasoning_scores': []
        }
        
        # Reasoning dataset
        self.reasoning_dataset = self._create_reasoning_dataset()
    
    def _create_reasoning_dataset(self) -> List[Dict[str, Any]]:
        """
        Create a simple reasoning dataset for training
        
        In practice, this would be a large, curated dataset
        """
        dataset = []
        
        # Math problems
        math_problems = [
            {
                'type': 'math',
                'problem': 'What is 5 + 3?',
                'ground_truth': '8',
                'difficulty': 1.0
            },
            {
                'type': 'math',
                'problem': 'Solve for x: 2x + 5 = 15',
                'ground_truth': 'x = 5',
                'difficulty': 2.0
            },
            {
                'type': 'math',
                'problem': 'What is the area of a rectangle with length 5 and width 3?',
                'ground_truth': '15',
                'difficulty': 1.5
            }
        ]
        
        # Logic problems
        logic_problems = [
            {
                'type': 'logic',
                'problem': 'All humans are mortal. Socrates is a human. What can we conclude?',
                'ground_truth': 'Socrates is mortal',
                'difficulty': 1.0
            },
            {
                'type': 'logic',
                'problem': 'If it rains, the ground gets wet. It is raining. What happens to the ground?',
                'ground_truth': 'The ground gets wet',
                'difficulty': 1.0
            }
        ]
        
        dataset.extend(math_problems)
        dataset.extend(logic_problems)
        
        return dataset
    
    def _sample_problem(self) -> Dict[str, Any]:
        """
        Sample a problem from the dataset based on current difficulty
        """
        # Get current difficulty level
        difficulty = self.ppo_agent.reasoning_difficulty
        
        # Filter problems by difficulty
        suitable_problems = [
            problem for problem in self.reasoning_dataset
            if abs(problem['difficulty'] - difficulty) <= 1.0
        ]
        
        if not suitable_problems:
            suitable_problems = self.reasoning_dataset
        
        return random.choice(suitable_problems)
    
    def _generate_reasoning_response(self, problem: str) -> Dict[str, Any]:
        """
        Simulate LLM reasoning response generation
        
        In practice, this would use the actual LLM policy
        """
        # Simple simulation based on problem type
        if '5 + 3' in problem:
            return {
                'reasoning_steps': [
                    'I need to add 5 and 3.',
                    '5 + 3 equals 8.',
                    'So the answer is 8.'
                ],
                'answer': '8'
            }
        elif '2x + 5 = 15' in problem:
            return {
                'reasoning_steps': [
                    'This is a linear equation.',
                    'First, subtract 5 from both sides: 2x = 10.',
                    'Then divide by 2: x = 5.',
                    'So the solution is x = 5.'
                ],
                'answer': 'x = 5'
            }
        elif 'area of a rectangle' in problem:
            return {
                'reasoning_steps': [
                    'Area of rectangle is length × width.',
                    'Given length = 5 and width = 3.',
                    'So area = 5 × 3 = 15.',
                    'The answer is 15.'
                ],
                'answer': '15'
            }
        elif 'Socrates' in problem:
            return {
                'reasoning_steps': [
                    'All humans are mortal.',
                    'Socrates is a human.',
                    'Therefore, Socrates is mortal.'
                ],
                'answer': 'Socrates is mortal'
            }
        elif 'rains' in problem:
            return {
                'reasoning_steps': [
                    'If it rains, the ground gets wet.',
                    'It is raining.',
                    'Therefore, the ground gets wet.'
                ],
                'answer': 'The ground gets wet'
            }
        else:
            # Default response
            return {
                'reasoning_steps': [
                    'I need to think about this problem.',
                    'Let me analyze it step by step.',
                    'The answer seems to be unknown.'
                ],
                'answer': 'unknown'
            }
    
    def train(self, verbose: bool = True) -> Dict[str, Any]:
        """
        Run the complete training process
        """
        print("Starting DeepSeek-R1 Training...")
        print(f"Training for {self.max_episodes} episodes...")
        
        for episode in range(1, self.max_episodes + 1):
            if verbose and episode % 10 == 0:
                print(f"\nEpisode {episode}/{self.max_episodes}")
            
            # Sample a problem
            problem = self._sample_problem()
            
            # Generate reasoning response (simulated)
            response = self._generate_reasoning_response(problem['problem'])
            
            # Calculate reasoning reward
            reward = self.ppo_agent.reward_calculator.calculate_total_reward(
                response['reasoning_steps'],
                response['answer'],
                problem['ground_truth']
            )
            
            # Store transition (simplified)
            state = {
                'problem': problem['problem'],
                'reasoning_steps': response['reasoning_steps'],
                'ground_truth': problem['ground_truth']
            }
            
            action = {
                'answer': response['answer'],
                'reasoning_steps': response['reasoning_steps']
            }
            
            # Simulate log probability and value
            log_prob = torch.tensor([0.0], device=device)  # Simplified
            value = 0.5  # Simplified
            
            # Store in memory
            self.ppo_agent.store_transition(
                state, action, log_prob, reward, True, value
            )
            
            # Evaluate reasoning
            evaluation = self.evaluator.evaluate_reasoning(
                response['reasoning_steps'],
                response['answer'],
                problem['ground_truth'],
                problem['type']
            )
            
            # Store metrics
            self.training_metrics['episode_rewards'].append(reward)
            self.training_metrics['answer_accuracy'].append(float(evaluation['answer_correct']))
            self.training_metrics['reasoning_scores'].append(evaluation['overall_score'])
            
            # Perform PPO update every few episodes
            if episode % 5 == 0 and len(self.ppo_agent.memory['states']) > 0:
                policy_loss, value_loss, reasoning_loss, total_loss = self.ppo_agent.ppo_update()
                
                self.training_metrics['policy_losses'].append(policy_loss)
                self.training_metrics['value_losses'].append(value_loss)
                self.training_metrics['reasoning_losses'].append(reasoning_loss)
                self.training_metrics['total_losses'].append(total_loss)
                
                if verbose:
                    print(f"  PPO Update - Policy Loss: {policy_loss:.4f}, " +
                          f"Value Loss: {value_loss:.4f}, " +
                          f"Reasoning Loss: {reasoning_loss:.4f}, " +
                          f"Total Loss: {total_loss:.4f}")
            
            # Show progress
            if verbose and episode % 10 == 0:
                avg_reward = np.mean(self.training_metrics['episode_rewards'][-10:])
                avg_accuracy = np.mean(self.training_metrics['answer_accuracy'][-10:])
                avg_reasoning_score = np.mean(self.training_metrics['reasoning_scores'][-10:])
                
                print(f"  Recent Performance - Avg Reward: {avg_reward:.3f}, " +
                      f"Accuracy: {avg_accuracy:.3f}, " +
                      f"Reasoning Score: {avg_reasoning_score:.3f}")
                
                # Show reasoning performance
                reasoning_perf = self.ppo_agent.get_reasoning_performance()
                print(f"  Reasoning Performance - Steps: {reasoning_perf['avg_steps']:.1f}, " +
                      f"Consistency: {reasoning_perf['avg_consistency']:.3f}, " +
                      f"Difficulty: {reasoning_perf['current_difficulty']:.2f}")
        
        print("\nTraining completed!")
        return self.get_training_summary()
    
    def get_training_summary(self) -> Dict[str, Any]:
        """
        Get summary of training results
        """
        summary = {
            'total_episodes': len(self.training_metrics['episode_rewards']),
            'avg_reward': np.mean(self.training_metrics['episode_rewards']),
            'max_reward': np.max(self.training_metrics['episode_rewards']),
            'avg_accuracy': np.mean(self.training_metrics['answer_accuracy']),
            'avg_reasoning_score': np.mean(self.training_metrics['reasoning_scores']),
            'avg_policy_loss': np.mean(self.training_metrics['policy_losses']) if self.training_metrics['policy_losses'] else 0,
            'avg_value_loss': np.mean(self.training_metrics['value_losses']) if self.training_metrics['value_losses'] else 0,
            'avg_reasoning_loss': np.mean(self.training_metrics['reasoning_losses']) if self.training_metrics['reasoning_losses'] else 0,
            'final_reasoning_performance': self.ppo_agent.get_reasoning_performance(),
            'evaluation_summary': self.evaluator.get_evaluation_summary()
        }
        
        return summary
    
    def plot_training_metrics(self):
        """
        Plot training metrics
        """
        plt.figure(figsize=(15, 12))
        
        # Episode rewards
        plt.subplot(3, 2, 1)
        plt.plot(self.training_metrics['episode_rewards'])
        plt.title('Episode Rewards Over Time')
        plt.xlabel('Episode')
        plt.ylabel('Reward')
        plt.grid(True)
        
        # Answer accuracy
        plt.subplot(3, 2, 2)
        plt.plot(self.training_metrics['answer_accuracy'])
        plt.title('Answer Accuracy Over Time')
        plt.xlabel('Episode')
        plt.ylabel('Accuracy')
        plt.ylim(0, 1.1)
        plt.grid(True)
        
        # Reasoning scores
        plt.subplot(3, 2, 3)
        plt.plot(self.training_metrics['reasoning_scores'])
        plt.title('Reasoning Scores Over Time')
        plt.xlabel('Episode')
        plt.ylabel('Score')
        plt.ylim(0, 1)
        plt.grid(True)
        
        # Policy losses
        if self.training_metrics['policy_losses']:
            plt.subplot(3, 2, 4)
            plt.plot(self.training_metrics['policy_losses'])
            plt.title('Policy Loss Over Time')
            plt.xlabel('Update')
            plt.ylabel('Loss')
            plt.grid(True)
        
        # Value losses
        if self.training_metrics['value_losses']:
            plt.subplot(3, 2, 5)
            plt.plot(self.training_metrics['value_losses'])
            plt.title('Value Loss Over Time')
            plt.xlabel('Update')
            plt.ylabel('Loss')
            plt.grid(True)
        
        # Reasoning losses
        if self.training_metrics['reasoning_losses']:
            plt.subplot(3, 2, 6)
            plt.plot(self.training_metrics['reasoning_losses'])
            plt.title('Reasoning Loss Over Time')
            plt.xlabel('Update')
            plt.ylabel('Loss')
            plt.grid(True)
        
        plt.tight_layout()
        plt.show()


### 5.2 Training Simulation

Let's run a simulation of the DeepSeek-R1 training process:

In [ ]:
# Create simple neural networks for demonstration
class SimplePolicyNetwork(torch.nn.Module):
    def __init__(self, input_size=10, output_size=5):
        super().__init__()
        self.fc = torch.nn.Linear(input_size, output_size)
        self.log_std = torch.nn.Parameter(torch.zeros(output_size))
    
    def forward(self, x):
        # Simple implementation for demonstration
        if isinstance(x, dict):
            # Convert problem to simple embedding
            problem_embedding = torch.tensor([len(x.get('problem', '')) / 100.0], device=device)
            problem_embedding = problem_embedding.expand(10)  # Expand to input size
        else:
            problem_embedding = torch.randn(10, device=device)
        
        mean = self.fc(problem_embedding)
        std = self.log_std.exp()
        
        # Create normal distribution
        dist = torch.distributions.Normal(mean, std)
        
        return dist
    
    def act(self, x):
        dist = self(x)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action, log_prob

class SimpleValueNetwork(torch.nn.Module):
    def __init__(self, input_size=10):
        super().__init__()
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(input_size, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 1)
        )
    
    def forward(self, x):
        if isinstance(x, dict):
            # Convert problem to simple embedding
            problem_embedding = torch.tensor([len(x.get('problem', '')) / 100.0], device=device)
            problem_embedding = problem_embedding.expand(10)  # Expand to input size
        else:
            problem_embedding = torch.randn(10, device=device)
        
        return self.fc(problem_embedding)

# Initialize components
policy_net = SimplePolicyNetwork()
value_net = SimpleValueNetwork()
reward_calc = AdvancedReasoningReward()
evaluator = ReasoningEvaluator()

# Create DeepSeek-R1 PPO agent
deepseek_ppo = DeepSeekR1PPO(
    policy_network=policy_net,
    value_network=value_net,
    reward_calculator=reward_calc,
    lr=1e-3,
    clip_epsilon=0.2,
    ppo_epochs=3,
    batch_size=4
)

# Create trainer
trainer = DeepSeekR1Trainer(
    ppo_agent=deepseek_ppo,
    evaluator=evaluator,
    max_episodes=30,  # Short training for demonstration
    max_steps_per_episode=10
)

# Run training
training_results = trainer.train(verbose=True)

# Show training summary
print("\n=== Training Summary ===")
for key, value in training_results.items():
    if isinstance(value, dict):
        print(f"\n{key}:")
        for sub_key, sub_value in value.items():
            if isinstance(sub_value, (float, int)):
                print(f"  {sub_key}: {sub_value:.4f}")
            else:
                print(f"  {sub_key}: {sub_value}")
    elif isinstance(value, (float, int)):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

# Plot training metrics
trainer.plot_training_metrics()

# Plot evaluation metrics
trainer.evaluator.plot_evaluation_metrics()

## 6. Nigerian Context Applications <a id='nigerian-apps'></a>

### 6.1 Nigerian-Specific Reasoning Applications

Let's explore how DeepSeek-R1 reasoning capabilities can be applied to Nigerian context:

In [ ]:
class NigerianReasoningApplications:
    """
    Nigerian context applications for DeepSeek-R1 reasoning capabilities
    """
    
    def __init__(self):
        self.reasoning_evaluator = ReasoningEvaluator()
        self.application_examples = self._create_nigerian_examples()
    
    def _create_nigerian_examples(self) -> List[Dict[str, Any]]:
        """
        Create Nigerian-specific reasoning examples
        """
        examples = []
        
        # Education - OAU student assessments
        examples.append({
            'domain': 'education',
            'application': 'OAU Student Assessments',
            'description': 'Enhanced reasoning for automated grading and feedback',
            'example_problem': {
                'problem': 'A student at OAU is studying Computer Science. They need to solve this algorithm problem: Given an array of integers, find the maximum subarray sum.',
                'ground_truth': 'Kadane\'s algorithm with O(n) time complexity',
                'reasoning_steps': [
                    'Understand the problem: find maximum subarray sum.',
                    'Consider brute force approach: O(n^2) time complexity.',
                    'Research efficient algorithms: Kadane\'s algorithm.',
                    'Implement Kadane\'s algorithm: O(n) time complexity.',
                    'Test with sample inputs to verify correctness.',
                    'Conclusion: Kadane\'s algorithm is optimal solution.'
                ],
                'answer': 'Kadane\'s algorithm with O(n) time complexity'
            }
        })
        
        # Healthcare - Medical diagnosis support
        examples.append({
            'domain': 'healthcare',
            'application': 'Medical Diagnosis Support',
            'description': 'Reasoning-enhanced medical decision support for Nigerian healthcare',
            'example_problem': {
                'problem': 'A patient presents with fever, headache, and joint pain. They live in a malaria-endemic area of Nigeria. What are the possible diagnoses and recommended tests?',
                'ground_truth': 'Malaria, dengue fever, typhoid fever; recommend malaria rapid diagnostic test and blood tests',
                'reasoning_steps': [
                    'Analyze symptoms: fever, headache, joint pain.',
                    'Consider geographical context: malaria-endemic area in Nigeria.',
                    'Differential diagnosis: malaria, dengue fever, typhoid fever.',
                    'Malaria is most likely due to endemic nature.',
                    'Recommend malaria rapid diagnostic test (RDT).',
                    'Also consider blood tests for complete blood count and typhoid screening.',
                    'If malaria RDT positive, start appropriate antimalarial treatment.',
                    'If negative, consider other tropical diseases common in Nigeria.'
                ],
                'answer': 'Malaria, dengue fever, typhoid fever; recommend malaria rapid diagnostic test and blood tests'
            }
        })
        
        # Agriculture - Crop optimization
        examples.append({
            'domain': 'agriculture',
            'application': 'Crop Optimization',
            'description': 'Reasoning for optimal farming decisions in Nigerian agriculture',
            'example_problem': {
                'problem': 'A farmer in Kaduna State wants to optimize maize yield. Current practices: traditional farming, rain-fed irrigation, no fertilizer. What recommendations would improve yield?',
                'ground_truth': 'Use improved seed varieties, apply NPK fertilizer, implement proper spacing, consider irrigation supplements',
                'reasoning_steps': [
                    'Analyze current practices: traditional farming, rain-fed, no fertilizer.',
                    'Research maize farming best practices for Nigerian context.',
                    'Consider soil testing for nutrient deficiencies.',
                    'Recommend improved seed varieties suitable for Kaduna climate.',
                    'Suggest NPK fertilizer application based on soil test results.',
                    'Recommend proper plant spacing for optimal growth.',
                    'Consider supplemental irrigation during dry periods.',
                    'Evaluate cost-benefit analysis for smallholder farmer context.'
                ],
                'answer': 'Use improved seed varieties, apply NPK fertilizer, implement proper spacing, consider irrigation supplements'
            }
        })
        
        # Governance - Policy analysis
        examples.append({
            'domain': 'governance',
            'application': 'Policy Analysis',
            'description': 'Reasoning-enhanced policy analysis for Nigerian governance',
            'example_problem': {
                'problem': 'Analyze the potential impact of removing fuel subsidies in Nigeria on inflation, transportation costs, and household budgets.',
                'ground_truth': 'Inflation increase, higher transportation costs, significant impact on low-income households, potential for social unrest',
                'reasoning_steps': [
                    'Understand current fuel subsidy system and its economic impact.',
                    'Analyze historical data on fuel price changes and inflation.',
                    'Examine transportation sector dependence on subsidized fuel.',
                    'Assess household budget allocation for transportation and energy.',
                    'Consider income distribution and poverty levels in Nigeria.',
                    'Evaluate potential social and political consequences.',
                    'Recommend mitigation strategies: targeted cash transfers, public transportation improvements.',
                    'Suggest phased implementation to minimize economic shock.'
                ],
                'answer': 'Inflation increase, higher transportation costs, significant impact on low-income households, potential for social unrest'
            }
        })
        
        # Education - Yoruba language processing
        examples.append({
            'domain': 'education',
            'application': 'Yoruba Language Processing',
            'description': 'Reasoning for Yoruba language understanding and generation',
            'example_problem': {
                'problem': 'Translate and explain this Yoruba proverb: "Adìe fúnfun ńgbè́ ńlẹ̀, tó bá dára, a á pe é ńgbè́ ńpupọ." What is the literal meaning and cultural significance?',
                'ground_truth': 'Literal: A white chicken that lays eggs quietly, if it is good, we will call it a chicken that lays many eggs. Cultural: Good deeds, even if done quietly, will be recognized and appreciated.',
                'reasoning_steps': [
                    'Analyze Yoruba proverb structure and vocabulary.',
                    'Break down sentence: Adìe fúnfun (white chicken), ńgbè́ ńlẹ̀ (lays eggs quietly).',
                    'Second part: tó bá dára (if it is good), a á pe é ńgbè́ ńpupọ (we will call it a chicken that lays many eggs).',
                    'Literal translation: A white chicken that lays eggs quietly, if it is good, we will call it a chicken that lays many eggs.',
                    'Cultural context: Yoruba values humility and quiet excellence.',
                    'Interpretation: Good deeds done quietly will be recognized.',
                    'Application: Encourages humility and consistent good behavior.'
                ],
                'answer': 'Literal: A white chicken that lays eggs quietly, if it is good, we will call it a chicken that lays many eggs. Cultural: Good deeds, even if done quietly, will be recognized and appreciated.'
            }
        })
        
        return examples
    
    def evaluate_nigerian_applications(self) -> Dict[str, Any]:
        """
        Evaluate reasoning performance on Nigerian applications
        """
        results = {}
        
        for example in self.application_examples:
            domain = example['domain']
            application = example['application']
            problem_data = example['example_problem']
            
            # Evaluate reasoning
            evaluation = self.reasoning_evaluator.evaluate_reasoning(
                problem_data['reasoning_steps'],
                problem_data['answer'],
                problem_data['ground_truth'],
                domain
            )
            
            # Store results
            results[f"{domain}_{application}"] = {
                'description': example['description'],
                'evaluation': evaluation,
                'problem': problem_data['problem'],
                'answer_correct': evaluation['answer_correct'],
                'overall_score': evaluation['overall_score']
            }
        
        return results
    
    def analyze_nigerian_impact(self) -> Dict[str, Any]:
        """
        Analyze potential impact of reasoning-enhanced LLMs in Nigerian context
        """
        results = self.evaluate_nigerian_applications()
        
        # Calculate overall performance
        total_score = sum(result['overall_score'] for result in results.values())
        avg_score = total_score / len(results)
        
        correct_answers = sum(1 for result in results.values() if result['answer_correct'])
        accuracy = correct_answers / len(results)
        
        # Domain-specific analysis
        domain_performance = {}
        for result_key, result_data in results.items():
            domain = result_key.split('_')[0]
            if domain not in domain_performance:
                domain_performance[domain] = {'scores': [], 'correct': 0, 'total': 0}
            
            domain_performance[domain]['scores'].append(result_data['overall_score'])
            domain_performance[domain]['total'] += 1
            if result_data['answer_correct']:
                domain_performance[domain]['correct'] += 1
        
        # Calculate domain averages
        for domain in domain_performance:
            domain_performance[domain]['avg_score'] = np.mean(domain_performance[domain]['scores'])
            domain_performance[domain]['accuracy'] = domain_performance[domain]['correct'] / domain_performance[domain]['total']
        
        return {
            'overall_performance': {
                'average_score': avg_score,
                'accuracy': accuracy,
                'total_applications': len(results)
            },
            'domain_performance': domain_performance,
            'detailed_results': results,
            'impact_analysis': self._generate_impact_analysis(avg_score, accuracy)
        }
    
    def _generate_impact_analysis(self, avg_score: float, accuracy: float) -> Dict[str, str]:
        """
        Generate impact analysis based on performance metrics
        """
        impact_level = "high" if avg_score > 0.8 else "medium" if avg_score > 0.6 else "low"
        
        analysis = {
            'overall_impact': f"Reasoning-enhanced LLMs show {impact_level} potential for Nigerian applications with average score of {avg_score:.2f} and accuracy of {accuracy:.2f}.",
            'education_impact': "Can significantly improve OAU student assessments, Yoruba language preservation, and educational content generation.",
            'healthcare_impact': "Potential to enhance medical diagnosis support, especially in rural areas with limited healthcare access.",
            'agriculture_impact': "Can provide data-driven farming recommendations to improve crop yields and food security.",
            'governance_impact': "Enables more sophisticated policy analysis for evidence-based decision making.",
            'challenges': "Requires adaptation to Nigerian context, local language support, and addressing digital divide issues.",
            'recommendations': "Focus on domain-specific fine-tuning, local language integration, and partnerships with Nigerian institutions."
        }
        
        if avg_score > 0.8:
            analysis['recommendation'] = "Ready for pilot deployment in specific domains with high performance."
        elif avg_score > 0.6:
            analysis['recommendation'] = "Needs further training and adaptation before deployment."
        else:
            analysis['recommendation'] = "Requires significant improvement before practical application."
        
        return analysis
    
    def display_nigerian_applications(self):
        """
        Display Nigerian application examples and evaluations
        """
        print("🌍 Nigerian Context Applications for DeepSeek-R1 Reasoning")
        print("=" * 60)
        
        # Evaluate applications
        results = self.evaluate_nigerian_applications()
        
        for app_key, app_data in results.items():
            print(f"\n📋 {app_data['description']}")
            print(f"Domain: {app_key.split('_')[0].capitalize()}")
            print(f"Application: {app_key.split('_', 1)[1]}")
            print(f"\nProblem: {app_data['problem'][:100]}...")
            print(f"Answer Correct: {'✅' if app_data['answer_correct'] else '❌'}")
            print(f"Overall Score: {app_data['overall_score']:.3f}")
            print(f"Reasoning Quality: {app_data['evaluation']['reasoning_quality']:.3f}")
            print(f"Structure Score: {app_data['evaluation']['logical_structure']['structure_score']:.3f}")
            print("-" * 60)
        
        # Show impact analysis
        impact_analysis = self.analyze_nigerian_impact()
        
        print("\n📊 Impact Analysis Summary")
        print("=" * 60)
        print(f"Overall Average Score: {impact_analysis['overall_performance']['average_score']:.3f}")
        print(f"Overall Accuracy: {impact_analysis['overall_performance']['accuracy']:.3f}")
        print(f"\nImpact Assessment: {impact_analysis['impact_analysis']['overall_impact']}")
        
        print("\n🌱 Domain-Specific Performance:")
        for domain, perf in impact_analysis['domain_performance'].items():
            print(f"  {domain.capitalize()}: Score={perf['avg_score']:.3f}, Accuracy={perf['accuracy']:.3f}")
        
        print(f"\n💡 Recommendation: {impact_analysis['impact_analysis']['recommendation']}")


In [ ]:
# Explore Nigerian applications
nigerian_apps = NigerianReasoningApplications()
nigerian_apps.display_nigerian_applications()

# Get detailed impact analysis
impact_analysis = nigerian_apps.analyze_nigerian_impact()

print("\n📋 Detailed Impact Analysis by Domain:")
for domain, analysis in impact_analysis['impact_analysis'].items():
    if domain != 'recommendation':
        print(f"\n{domain.replace('_', ' ').title()}: {analysis}")

print(f"\n🎯 Final Recommendation: {impact_analysis['impact_analysis']['recommendation']}")

## 🎓 Summary and Next Steps

### 7.1 Implementation Summary

In this notebook, we have implemented the core components of DeepSeek-R1:

1. **Reasoning-Centric Reward Design**:
   - Answer correctness evaluation
   - Reasoning path quality assessment
   - Logical consistency checking
   - Reasoning efficiency metrics
   - Advanced metrics (novelty, depth)

2. **PPO Implementation for Reasoning**:
   - Basic PPO algorithm
   - Reasoning-specific adaptations
   - Curriculum learning
   - Reasoning structure regularization
   - Performance metrics tracking

3. **Comprehensive Evaluation Framework**:
   - Multi-dimensional reasoning assessment
   - Logical structure analysis
   - Mathematical rigor evaluation
   - Creativity metrics
   - Performance visualization

4. **Complete Training Pipeline**:
   - Reasoning dataset creation
   - Training simulation
   - Performance monitoring
   - Nigerian context applications

5. **Nigerian Context Applications**:
   - Education (OAU assessments, Yoruba language)
   - Healthcare (medical diagnosis)
   - Agriculture (crop optimization)
   - Governance (policy analysis)
   - Impact assessment

### 7.2 Key Insights

**Reasoning-Centric RL Advantages:**
- Enables optimization for complex reasoning tasks beyond simple pattern matching
- Provides fine-grained control over reasoning quality
- Can adapt to specific domains and cultural contexts
- Offers interpretability through reasoning step analysis

**Challenges Addressed:**
- Reward function design for complex reasoning
- Stable training with reasoning-specific regularization
- Multi-dimensional evaluation of reasoning quality
- Curriculum learning for progressive difficulty

**Nigerian Context Potential:**
- Significant impact across education, healthcare, agriculture, and governance
- Addresses specific Nigerian challenges and opportunities
- Supports local language preservation and cultural context understanding
- Can bridge gaps in access to expertise

### 7.3 Next Steps for Implementation

1. **Enhance Reward Functions**:
   - Integrate semantic similarity for answer evaluation
   - Add domain-specific reasoning metrics
   - Incorporate knowledge graph validation

2. **Improve PPO Implementation**:
   - Implement full LLM policy integration
   - Add attention mechanism analysis
   - Enhance curriculum learning algorithms

3. **Expand Evaluation Framework**:
   - Add benchmark dataset integration
   - Implement human evaluation correlation
   - Develop domain-specific evaluation metrics

4. **Nigerian Context Development**:
   - Create Nigerian-specific reasoning datasets
   - Develop Yoruba and other local language support
   - Partner with Nigerian institutions for real-world testing

5. **Deployment and Scaling**:
   - Optimize for edge deployment in low-resource settings
   - Develop API interfaces for integration
   - Create user-friendly interfaces for non-technical users

### 7.4 Research Directions

1. **Reasoning Evaluation Metrics**:
   - Develop more sophisticated reasoning quality metrics
   - Create automated reasoning structure analysis
   - Build correlation with human expert evaluations

2. **Multi-Modal Reasoning**:
   - Extend to text + image reasoning
   - Incorporate structured data reasoning
   - Develop cross-modal reasoning evaluation

3. **Cultural Adaptation**:
   - Study cultural differences in reasoning patterns
   - Develop culturally-adaptive reasoning models
   - Create local language reasoning benchmarks

4. **Ethical Considerations**:
   - Address bias in reasoning evaluation
   - Ensure fairness across different contexts
   - Develop responsible deployment guidelines

### 7.5 Conclusion

This implementation demonstrates the core principles of DeepSeek-R1 and provides a foundation for reasoning-centric reinforcement learning. The approach shows significant promise for enhancing LLM reasoning capabilities, particularly in context-specific applications like those relevant to Nigeria.

The next phase would involve:
- Integration with actual LLM architectures
- Large-scale training on diverse reasoning datasets
- Real-world testing and validation
- Continuous improvement through human feedback

This work represents an important step toward developing AI systems that can reason effectively about complex problems while being adaptable to specific cultural and contextual needs.